### 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助您：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(flyai_agent_in_action)`，即可执行下方代码

In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate flyai_agent_in_action

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "flyai_agent_in_action" ]; then
    echo "✅ 当前单元格已成功激活到 flyai_agent_in_action 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的 Python 单元格将使用 Notebook 当前选择的 Jupyter 内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(flyai_agent_in_action)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(flyai_agent_in_action)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate flyai_agent_in_action"
fi

echo "=========================================" 

== Conda 环境检查报告 (仅针对当前 Bash 子进程) ==
✅ 当前单元格已成功激活到 flyai_agent_in_action 环境。
✅ 正在使用的环境路径: /workspace/envs/flyai_agent_in_action

💡 提示: 后续的 Python 单元格将使用 Notebook 当前选择的 Jupyter 内核。
   如果需要后续单元格也使用此环境，请执行以下操作:
   1. 检查 Notebook 右上角是否已选择 'python(flyai_agent_in_action)'。


In [2]:
# 2. 设置pip 为清华源
%pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list


Writing to /root/.config/pip/pip.conf
Note: you may need to restart the kernel to use updated packages.
global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'
:env:.target=''
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT

env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [4]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                  |
|:-------------|:----------------------------------------------------------------------|
| 操作系统     | Linux 5.15.0-126-generic                                              |
| CPU 信息     | Intel(R) Xeon(R) Platinum 8468 (48 physical cores, 192 logical cores) |
| 内存信息     | 2015.36 GB (Available: 1868.62 GB)                                    |
| GPU 信息     | No GPU found (checked nvidia-smi, lshw not found)                     |
| CUDA 信息    | 12.6                                                                  |
| Python 版本  | 3.12.11                                                               |
| Conda 版本   | conda 25.7.0                                                          |
| 物理磁盘空间 | Total: 2014.78 GB, Used: 788.88 GB, Free: 1123.48 GB                  |


In [5]:
# 查看当前使用的Coda环境（带有*）
%conda env list


# conda environments:
#
base                   /opt/conda
lf                     /opt/conda/envs/lf
flyai_agent_in_action * /workspace/envs/flyai_agent_in_action


Note: you may need to restart the kernel to use updated packages.



## LangGraph教程：LangChain


-----

### **背景介绍**

在 LangChain，我们的目标是让构建大型语言模型（LLM）应用变得简单。您可以构建的一种 LLM 应用就是**智能代理（Agent）**。构建智能代理非常令人兴奋，因为它们能够自动化以前不可能完成的各种任务。

然而，在实践中，构建能够可靠执行这些任务的系统极其困难。在我们与用户合作将智能代理投入生产的过程中，我们发现通常需要更多的**控制**。例如，您可能需要智能代理始终优先调用某个特定的工具，或者根据其状态使用不同的提示词。

为了解决这个问题，我们构建了 [**LangGraph**](https://langchain-ai.github.io/langgraph/) —— 一个用于构建智能代理和多智能体应用的框架。LangGraph 独立于 LangChain 包，其核心设计理念是帮助开发者为智能代理工作流添加更好的**精确性**和**控制力**，使其适合现实世界系统的复杂性。

-----

### **设置**

在开始之前，请按照 `README` 文件中的说明创建环境并安装依赖项。

-----

### **聊天模型**

在本课程中，我们将使用[**聊天模型（Chat Models）**](https://python.langchain.com/v0.2/docs/concepts/#chat-models)，它们的功能是接收一系列消息作为输入，并以聊天消息作为输出。LangChain 本身不托管任何聊天模型，而是依赖于第三方集成。 是 LangChain 中支持的第三方聊天模型集成列表！默认情况下，课程将使用[ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/)  ，因为它既流行又性能出色。正如前面提到的，请确保您已设置好 `OPENAI_API_KEY`。

我们将检查您的 `OPENAI_API_KEY` 是否已设置；如果未设置，系统会提示您输入。


#### OpenAI国内代理

为方便大家再国内使用OpenAI，Claude等国外模型

和OpenAI国内代理API易社区争取到了如下权益：

OpenAI国内代理地址
https://api.apiyi.com/register/?aff_code=we80
新用户注册送0.1美金，注册成功后在以下表格中填写你的账号，平台会再赠送2美金（5个工作日到账）

---- 【腾讯文档】API易账号收集 https://docs.qq.com/form/page/DQm1qb1VBQU9wR2xq


In [6]:
# 安装必要的依赖包
# %pip install 是 Jupyter 中安装 Python 包的命令
# --quiet 参数减少输出信息
%pip install --quiet langchain_openai==0.3.32 langchain_core==0.3.75 langchain_community==0.3.29 tavily-python==0.7.11

Note: you may need to restart the kernel to use updated packages.


In [7]:
# 导入必要的模块
import os, getpass

def _set_env(var: str):
    """
    设置环境变量的辅助函数

    参数:
        var (str): 要设置的环境变量名称

    功能:
        - 检查环境变量是否已存在
        - 如果不存在，则提示用户输入并设置
    """
    if not os.environ.get(var):  # 检查环境变量是否已设置
        os.environ[var] = getpass.getpass(f"{var}: ")  # 安全地获取用户输入

# 设置 OpenAI API 密钥
# 这是使用 OpenAI 模型所必需的
_set_env("OPENAI_API_KEY")
# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")


OPENAI_API_KEY:  ········
OPENAI_BASE_URL:  ········


[这里](https://python.langchain.com/v0.2/docs/how_to/#chat-models)是一个有用的指南，介绍了您可以使用聊天模型做的所有事情，但我们在下面会展示一些重点。如果您按照 README 中的说明运行了 `pip install -r requirements.txt`，那么您已经安装了 `langchain-openai` 包。有了这个包，我们可以实例化我们的 `ChatOpenAI` 模型对象。如果您是第一次注册 API，您应该会收到[免费积分](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517)，可以应用于任何模型。您可以[在这里](https://openai.com/api/pricing/)查看各种模型的定价。笔记本将默认使用 `gpt-4o`，因为它在质量、价格和速度之间取得了良好的平衡[更多信息请参见这里](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini)，但您也可以选择价格较低的 `gpt-3.5` 系列模型。

聊天模型有几个[标准参数](https://python.langchain.com/v0.2/docs/concepts/#chat-models)可以设置。最常见的两个是：

* `model`：模型名称
* `temperature`：采样温度

`Temperature` 控制模型输出的随机性或创造性，其中低温度（接近 0）产生更确定性和专注的输出。这适合需要准确性或事实性响应的任务。高温度（接近 1）适合创造性任务或生成多样化响应。

In [8]:
# 导入 ChatOpenAI 类
from langchain_openai import ChatOpenAI

# 创建 GPT-4o 聊天模型实例
# model="gpt-4o": 使用 GPT-4o 模型，这是 OpenAI 的最新模型
# temperature=0: 设置温度为 0，使输出更加确定性和一致
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)

# 创建 GPT-3.5 Turbo 聊天模型实例
# model="gpt-3.5-turbo-0125": 使用 GPT-3.5 Turbo 模型
# temperature=0: 同样设置为 0，确保输出的一致性
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

LangChain 中的聊天模型有许多[默认方法](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface)。在大多数情况下，我们将使用：

* `stream`：流式返回响应的块
* `invoke`：在输入上调用链

如前所述，聊天模型接受[消息](https://python.langchain.com/v0.2/docs/concepts/#messages)作为输入。消息具有一个角色（描述谁在说消息）和一个内容属性。我们稍后会更多地讨论这个问题，但这里让我们先展示基础知识。

In [9]:
# 导入 HumanMessage 类，用于创建人类用户的消息
from langchain_core.messages import HumanMessage

# 创建一个人类消息
# content: 消息内容
# name: 发送者的名称（可选）
msg = HumanMessage(content="你好呀", name="萤火AI百宝箱")

# 创建消息列表
# 聊天模型通常接受消息列表作为输入
messages = [msg]

# 使用消息列表调用模型
# invoke() 方法会发送消息给模型并返回响应
gpt4o_chat.invoke(messages)

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 17, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'input_tokens': 0, 'output_tokens': 0, 'input_tokens_details': None}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_5d7ee1b844', 'id': 'chatcmpl-CMRCwguhduNHehBqxpJWrY6sUklau', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c5c7842f-328c-47ce-aa90-8845c2cf9eff-0', usage_metadata={'input_tokens': 17, 'output_tokens': 10, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

我们得到一个 `AIMessage` 响应。另外，请注意我们可以直接用字符串调用聊天模型。当字符串作为输入传递时，它会被转换为 `HumanMessage`，然后传递给底层模型。


In [10]:
# 直接使用字符串调用模型
# 当传入字符串时，LangChain 会自动将其转换为 HumanMessage
gpt4o_chat.invoke("通俗解释LangGraph?包含应用场景，其他竞品对比分析，企业落地")

AIMessage(content='LangGraph 是一种用于处理和分析语言数据的技术框架或工具集。它通常结合了图形数据库和自然语言处理（NLP）技术，以便更有效地理解和处理复杂的语言结构和关系。以下是对 LangGraph 的通俗解释、应用场景、竞品对比分析以及企业落地的概述：\n\n### 通俗解释\nLangGraph 可以被视为一种将语言数据转化为图形结构的技术。图形结构能够更好地表示语言中的实体及其关系，比如词语之间的关联、句子结构、语义网络等。通过这种方式，LangGraph 可以帮助计算机更好地理解和处理自然语言。\n\n### 应用场景\n1. **知识图谱构建**：LangGraph 可以用于构建知识图谱，将文本数据转化为结构化的信息网络，帮助企业更好地管理和利用知识。\n   \n2. **信息检索与问答系统**：通过图形结构，LangGraph 可以提高信息检索的准确性和效率，支持更复杂的问答系统。\n\n3. **情感分析与舆情监测**：利用语言图谱分析社交媒体或新闻中的情感和舆情，帮助企业进行市场分析和品牌管理。\n\n4. **文本生成与翻译**：通过理解语言结构，LangGraph 可以提高机器翻译和文本生成的质量。\n\n### 竞品对比分析\nLangGraph 的竞品主要包括其他语言处理和图形数据库技术，如：\n\n- **Neo4j**：一种流行的图形数据库，虽然不专注于语言处理，但可以与 NLP 工具结合使用。\n- **GraphDB**：专注于语义图谱和知识管理，适合与语言处理结合。\n- **SpaCy 和 NLTK**：这些是流行的 NLP 库，可以与图形数据库结合使用以实现类似的功能。\n\nLangGraph 的优势在于其专注于语言数据的图形化处理，可能在处理复杂语言关系时更具优势。\n\n### 企业落地\n企业可以通过以下方式将 LangGraph 技术落地：\n\n1. **数据集成**：将企业现有的文本数据集成到 LangGraph 中，构建语言图谱以支持各种应用。\n\n2. **定制解决方案**：根据企业需求，定制化开发信息检索、问答系统或情感分析工具。\n\n3. **培训与支持**：提供员工培训和技术支持，帮助企业更好地利用 LangGraph 技术。\n\n4. **持续优化**：通过不断更

In [11]:
# 使用 GPT-3.5 Turbo 模型调用
# 同样可以直接传入字符串
gpt35_chat.invoke("通俗解释LangGraph?包含应用场景，其他竞品对比分析，企业落地")

AIMessage(content='LangGraph是一种基于语言学的图数据库，用于存储和查询语言相关的数据。它可以帮助用户更有效地管理和分析大量的语言数据，如文本、语音、图像等。\n\n应用场景：\n1. 语言学研究：研究人员可以利用LangGraph存储和分析大量的语言数据，从而更好地理解语言的结构和演化规律。\n2. 自然语言处理：LangGraph可以用于构建自然语言处理模型，帮助机器理解和生成自然语言。\n3. 语言教育：教育机构可以利用LangGraph存储和管理语言学习资源，为学生提供个性化的学习体验。\n\n竞品对比分析：\n与传统的关系型数据库相比，LangGraph更适合存储和查询语言相关的数据，因为它能够更好地处理语言之间的复杂关系。与其他图数据库相比，LangGraph专注于语言学领域，提供了更多针对语言数据的特定功能和优化。\n\n企业落地：\n企业可以利用LangGraph构建自己的语言数据平台，用于存储和分析大量的语言数据。例如，一家在线教育公司可以利用LangGraph存储学生的语言学习记录，并根据学生的学习情况提供个性化的学习建议。另外，一家语言学研究机构可以利用LangGraph存储和分析大量的语言数据，从而推动语言学研究的进展。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 447, 'prompt_tokens': 36, 'total_tokens': 483, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': None, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': None}, 'input_tokens': 0, 'output_tokens': 0, 'input_tokens_details': None}, 'model_name': 'gpt-3.

In [12]:
# 调用 stream 方法并遍历返回的流
stream_response = gpt35_chat.stream("通俗解释LangGraph?包含应用场景，其他竞品对比分析，企业落地")

# 打印每个分块的内容，并把它们连接起来形成完整回复
full_response = ""
for chunk in stream_response:
    # 提取每个分块（chunk）中的内容（content）
    content = chunk.content
    print(content, end="", flush=True) # 使用 print 实时输出内容
    full_response += content

print("\n--- 流式输出结束 ---")
# print(f"\n完整回复:\n{full_response}") # 打印完整回复

LangGraph是一种基于语言学的图数据库，用于存储和查询语言相关的数据。它可以帮助用户更有效地管理和分析大量的语言数据，如文本、语音、图像等。

应用场景：
1. 自然语言处理：LangGraph可以用于构建自然语言处理系统，帮助用户处理文本数据、进行语义分析、实现机器翻译等任务。
2. 语言学研究：研究人员可以利用LangGraph存储和分析语言学数据，探索语言结构、语言演化等问题。
3. 智能客服：LangGraph可以用于构建智能客服系统，帮助企业提供更高效的客户服务。

竞品对比分析：
与传统的关系型数据库相比，LangGraph具有更好的存储和查询效率，特别适用于存储和处理大规模的语言数据。与其他图数据库相比，LangGraph更专注于语言相关的数据，具有更丰富的语言学功能和特性。

企业落地：
企业可以通过部署LangGraph来构建自己的语言处理系统，提高数据管理和分析的效率。LangGraph可以与现有的数据处理工具和系统集成，帮助企业更好地利用语言数据实现商业目标。企业可以根据自身需求定制LangGraph的功能和性能，实现个性化的解决方案。
--- 流式输出结束 ---


所有聊天模型的接口都是一致的，模型通常在每个笔记本启动时初始化一次。

因此，如果您强烈偏好另一个提供商，您可以轻松地在模型之间切换，而无需更改下游代码。


## 搜索工具

您还会在 README 中看到 [Tavily](https://tavily.com/)，这是一个为 LLM 和 RAG 优化的搜索引擎，旨在提供高效、快速和持久的搜索结果。如前所述，注册很容易，并提供慷慨的免费层级。一些课程（在模块 4 中）将默认使用 Tavily，但当然，如果您想为自己修改代码，也可以使用其他搜索工具。

**Tavily API 密钥 (API Key) 的注册和申请流程**通常非常直接，但具体步骤可能会有细微变化。以下是一般性的指导和您可能需要遵循的步骤：

---

#### 1. 访问 Tavily 官方网站

首先，您需要访问 **Tavily 的官方网站**（您已经提供了链接：[https://tavily.com/](https://tavily.com/)）。

---

#### 2. 查找注册/登录或 API 页面

在网站上，您需要找到以下选项之一：
* **“Sign Up”（注册）** 或 **“Log In”（登录）** 按钮。
* **“API”、“Developers”（开发者）** 或 **“Pricing”（定价）** 页面。

通常，您需要先创建一个用户账户才能申请 API 密钥。

---

## 3. 注册新账户

如果您是新用户：
* 点击 **“Sign Up”** 或 **“注册”**。
* 您可能需要提供一个 **电子邮件地址** 和设置一个 **密码**。
* 有些服务也支持使用 **Google** 或其他第三方账户直接登录/注册。
* 完成注册后，您可能需要**验证您的邮箱**（通过点击发送到您邮箱的链接）。

---

#### 4. 访问 API 仪表板/设置

登录您的账户后，您需要进入到管理 API 密钥的区域，这通常被称为：
* **“Dashboard”（仪表板）**
* **“API Keys”（API 密钥）**
* **“Settings”（设置）** 或 **“Profile”（个人资料）**

---

#### 5. 生成 API 密钥

在这个 API 密钥管理页面：
* 查找类似于 **“Create New Key”（创建新密钥）**、**“Generate API Key”（生成 API 密钥）** 或 **“Get Started”（开始使用）** 的按钮。
* 点击该按钮，系统会**立即生成**一串独特的长字符，这就是您的 **Tavily API Key**。

---

## 6. 保存您的 API 密钥

**重要提示：**
* 生成的 API 密钥通常只会在**生成时显示一次**。
* **请务必立即复制并安全地存储**（例如，在密码管理器或安全文档中）您的密钥。
* 如果丢失，您可能需要生成一个新的密钥，而旧的密钥可能会被吊销。

完成这些步骤后，您就可以在您的应用程序或开发环境中使用这个 **TAVILY\_API\_KEY** 来调用 Tavily 的搜索 API 了。

In [13]:
# 设置 Tavily API 密钥
# 这是使用 Tavily 搜索功能所必需的
_set_env("TAVILY_API_KEY")

TAVILY_API_KEY:  ········


In [14]:
# 导入 Tavily 搜索结果工具
from langchain_community.tools.tavily_search import TavilySearchResults

# 创建 Tavily 搜索实例
# max_results=3: 设置最大返回结果数量为 3
tavily_search = TavilySearchResults(max_results=3)

# 执行搜索
# 搜索 "What is LangGraph?" 并获取结果
search_docs = tavily_search.invoke("通俗解释LangGraph?包含应用场景，其他竞品对比分析，企业落地")

/workspace/tmp/ipykernel_2477/4221156914.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=3)


In [15]:
# 显示搜索结果
# 这将输出搜索到的文档列表，包含 URL 和内容摘要
search_docs

[{'title': '【系统学习02】工作流vs Multi-Agent：Dify、LangGraph - 知乎专栏',
  'url': 'https://zhuanlan.zhihu.com/p/1950863538311627763',
  'content': 'Image 2: 血戎 本文把自己在 **Dify**、**LangGraph**、**AgentScope** 这三套工具上的实战感受放在一起对比，顺便聊聊 Multi-Agent 会不会像微服务那样走向“工程化”的未来。 | Dify (v1.6+) | DAG + 可视化编辑器 | 支持 Prompt 模板的变量替换 {{var}}，内置知识库检索（支持多源数据），工具调用需配置 OpenAPI 文档。 注意：内置的”Agent 节点”实际是工作流嵌套，非真正的 Multi-Agent | 上手快、低代码、适合业务流程 | 灵活性一般，多Agent弱（仅支持嵌套，无原生通信机制） | Multi-Agent 成功的关键不在于“有多少 Agent”，而在于 **通信方式、共享状态与角色分工**。 三大平台的 Multi-Agent 能力对比 | 平台 | 通信机制 | 多Agent支持 | 记忆 | 协作方式 | | 维度 | 工作流 | Multi-Agent | *   **Multi-Agent**：适合需要多角度判断、动态任务分解或角色并发协作的任务（例如复杂的市场分析、多角色对话系统、模拟谈判）。优点是灵活、能并行探索；缺点是调试和治理成本高。  2.   **可观测性**：日志、链路追踪、可视化监控会成为 Multi-Agent 平台标配（如：LangSmith）。 *   **方案**：**工作流**（流程清晰，各环节固定，无需多 Agent 协作） *   **方案**：**Multi-Agent**（需多角色协作，动态规划） *   **Multi-Agent**是你请来的一群网红大厨，灵感爆棚，但有时容易吵起来。 **未来趋势**：工作流和 Multi-Agent 不是非此即彼，而是互补共存。真正的高手，往往会两手都要抓——在流程性任务中用工作流保证稳定性，在探索性任务中用 Multi-Agent激发创造力。 **最终建议**：不要过早站队。先用工作流快速验证业务逻辑，再逐